In [1]:
import sys
import os

sys.path.append(os.path.abspath('..'))

import yaml
import pandas as pd

from src.data.load import load_data
from src.data.prepare import prepare_data
from src.models.cv_iterator import leave_last_k

In [2]:
# read config
with open('../config.yml', 'r') as file:
    config=yaml.load(file, Loader= yaml.SafeLoader)
del file

In [3]:
# load and prepare data
dfs = load_data(config=config['data_loader'])
#dfs = prepare_data(dataframes=dfs)

In [4]:
# train-test split
df_train, df_test = leave_last_k(df=dfs['data'], config=config['optimization'])
del df_test

In [5]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules

df = df_train[df_train['rating']>=4].copy()
df = df.groupby('user_id')['item_id'].apply(list).tolist()
te = TransactionEncoder()
df = te.fit(df).transform(df)
df = pd.DataFrame(df, columns=te.columns_)

frequent_itemsets = fpgrowth(df, min_support=0.1, use_colnames=True)
del df
frequent_itemsets

,support,itemsets
0,0.502123,(50)
1,0.417197,(100)
2,0.381104,(181)
3,0.358811,(174)
4,0.354565,(127)
...,...,...
2767,0.142251,"(50, 117, 181)"
2768,0.107219,"(50, 117, 174)"
2769,0.104034,"(50, 172, 117)"
2770,0.113588,"(50, 405)"


In [6]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.2)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(50),(100),0.502123,0.417197,0.265393,0.528541,1.266885,1.0,0.055908,1.236169,0.423122,0.405844,0.191049,0.582337
1,(100),(50),0.417197,0.502123,0.265393,0.636132,1.266885,1.0,0.055908,1.368291,0.361464,0.405844,0.269161,0.582337
2,(50),(181),0.502123,0.381104,0.340764,0.678647,1.780739,1.0,0.149403,1.925907,0.880610,0.628180,0.480764,0.786399
3,(181),(50),0.381104,0.502123,0.340764,0.894150,1.780739,1.0,0.149403,4.703626,0.708415,0.628180,0.787398,0.786399
4,(100),(181),0.417197,0.381104,0.195329,0.468193,1.228519,1.0,0.036333,1.163761,0.319167,0.323944,0.140717,0.490364
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16713,(117),"(50, 172)",0.238854,0.274947,0.104034,0.435556,1.584144,1.0,0.038362,1.284543,0.484459,0.253886,0.221513,0.406967
16714,(50),(405),0.502123,0.153928,0.113588,0.226216,1.469622,1.0,0.036297,1.093421,0.641831,0.209393,0.085439,0.482073
16715,(405),(50),0.153928,0.502123,0.113588,0.737931,1.469622,1.0,0.036297,1.899793,0.377690,0.209393,0.473627,0.482073
16716,(405),(181),0.153928,0.381104,0.101911,0.662069,1.737239,1.0,0.043248,1.831427,0.501581,0.235294,0.453978,0.464739
